In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import struct
from matplotlib.backends.backend_pdf import PdfPages

In [89]:
data = pd.read_csv('./twoStage.csv')

In [90]:
data.head()

,Unnamed: 0,id1,id2,ind,sampleTime,angleOffset,codes,6,7,8,9
0,0,787564065,431378677,-1,2.5,0,0xC0180F41C0420C4140CE0F41C09A174140F3154140D7...,NaN,NaN,NaN,NaN
1,1,787564064,431378677,-2,2.5,0,0x00000000AE6AD93F66283440BC1978405DBFA1401CE9...,NaN,NaN,NaN,NaN
2,2,787564035,431378662,-1,2.0,0,0x0040174180072841C0DF2C41C04E3F41C0244241C0E7...,NaN,NaN,NaN,NaN
3,3,787564033,431378662,-2,2.0,0,0x000000008FAD3E40C0759E408224DD4094CD0D41CBD0...,NaN,NaN,NaN,NaN
4,4,784722411,428368464,-1,2.0,0,0x4059204140052641C01B3A4140DA42418063444140F1...,NaN,NaN,NaN,NaN


In [91]:
tmp_data = data.sort_values(by='id2')

In [92]:
grouped = data.groupby('id2').size()

In [93]:
drop_data_id = grouped[grouped < 2].index.values
drop_data_id

array([431382425, 431438044, 453807557, 468726037])

In [94]:
for i in drop_data_id:
    tmp_data.drop(tmp_data[tmp_data.id2 == i].index, inplace=True)

In [95]:
new_grouped = tmp_data.groupby('id2').size()
data_id = new_grouped[grouped == 2].index.values
data_id#有效曲线图

array([420649411, 420649421, 427868971, 427868978, 428368464, 428368522,
       428368554, 428368575, 428368582, 428368612, 431052356, 431378662,
       431378677, 431378688, 431378711, 431378722, 431378740, 431382442,
       431383863, 431383901, 431383929, 431384022, 431384036, 431384049,
       431384094, 431385613, 431488349, 431495829, 453802654, 453802687,
       453802695, 453802738, 453802794, 453802814, 453802850, 453807397,
       468539387, 468539451, 468539523, 468539603, 468539655, 468539732,
       478408913, 478410853, 483588419, 483588506, 483591029, 483591069])

In [96]:
tmp_data.head()

,Unnamed: 0,id1,id2,ind,sampleTime,angleOffset,codes,6,7,8,9
16,16,779996558,420649411,-2,1.5,0,0x00000000C81A883F8516D93F73800B40A69830409A08...,NaN,NaN,NaN,NaN
15,15,779996560,420649411,-1,1.5,0,0x4078494100A78741A016A84160A3C741205AE441C0B2...,NaN,NaN,NaN,NaN
14,14,779996579,420649421,-2,1.5,0,0x000000001EAAB73E8CC60F3F9816513F4F10833FC398...,NaN,NaN,NaN,NaN
13,13,779996581,420649421,-1,1.5,0,0x80C21241801F154100C71641C013184100161A41C054...,NaN,NaN,NaN,NaN
8,8,784424322,427868971,-1,1.5,0,0x80E7184100A6214100282A418002314100CF38418006...,NaN,NaN,NaN,NaN


In [97]:
import struct
part_data = tmp_data[tmp_data.id2 == 420649411]
code = part_data[part_data.ind == -1].values
len(code[0][6])
hexcode = code[0][6][2:10]
bts = bytes.fromhex(hexcode)
struct.unpack('<f',bts)[0]

12.59185791015625

In [98]:
tmp_data.head()

,Unnamed: 0,id1,id2,ind,sampleTime,angleOffset,codes,6,7,8,9
16,16,779996558,420649411,-2,1.5,0,0x00000000C81A883F8516D93F73800B40A69830409A08...,NaN,NaN,NaN,NaN
15,15,779996560,420649411,-1,1.5,0,0x4078494100A78741A016A84160A3C741205AE441C0B2...,NaN,NaN,NaN,NaN
14,14,779996579,420649421,-2,1.5,0,0x000000001EAAB73E8CC60F3F9816513F4F10833FC398...,NaN,NaN,NaN,NaN
13,13,779996581,420649421,-1,1.5,0,0x80C21241801F154100C71641C013184100161A41C054...,NaN,NaN,NaN,NaN
8,8,784424322,427868971,-1,1.5,0,0x80E7184100A6214100282A418002314100CF38418006...,NaN,NaN,NaN,NaN


In [125]:

def decode(code):
    '''
    codes解码函数
    '''
    result = []
    for i in range(2, len(code), 8):
        hexcode = code[i: i + 8]
        bts = bytes.fromhex(hexcode)
        result.append(struct.unpack('<f', bts)[0])
    return result

def split_data(data):
    """
    从数组中取得首个极大值点，极小值点，以及最大值，并且返回
    （首个极大值，首个极小值，最大值）
    """
    maximal_data = 0
    minimal_data = 0
    max_data = 0
    
    threshold = 0.005
    
    for i in range(len(data)):
        if data[i] > data[max_data]:
            max_data = i
        if i > 0 and i < len(data) - 1: 
            if minimal_data == 0 and data[i+1] - data[i] > threshold * data[i]  and data[i-1] - data[i] > threshold* data[i]:
                minimal_data = i
            if maximal_data == 0 and data[i+1] - data[i] < -threshold * data[i] and data[i-1] - data[i] < -threshold * data[i]:
                maximal_data = i
        
    return (maximal_data, minimal_data, max_data)

def draw_pdf(id2_num):
    '''
    解码并画图
    '''
    part_data = tmp_data[tmp_data.id2 == id2_num]
    torque_code = part_data[part_data.ind == -1].values[0][6]
    angle_code = part_data[part_data.ind == -2].values[0][6]
    torque_list = decode(torque_code)
    angle_list = decode(angle_code)
    sample_time = part_data[part_data.ind == -1].values[0][4]
    time_list = [sample_time * i for i in range(len(torque_list))]
    
    angle_split = split_data(angle_list)
    torque_split = split_data(torque_list)
    
    #print(torque_split)
    
    with PdfPages(str(id2_num) + '.pdf') as pdf:
        plt.plot(angle_list, torque_list, label="0")
        plt.axvline(angle_list[torque_split[0]], color='r')
        plt.axvline(angle_list[torque_split[1]], color='g')
        plt.axvline(angle_list[torque_split[2]], color='y')
        plt.xlabel('Angle')
        plt.ylabel('Torque')
        plt.title("torque vs angle, id2 = %s"%(id2_num))
        pdf.savefig()
        plt.close()

        plt.plot(time_list, angle_list, label="angle")
        plt.plot(time_list, torque_list, label="torque")
        plt.axvline(sample_time * torque_split[0], color='r')
        plt.axvline(sample_time * torque_split[1], color='g')
        plt.axvline(sample_time * torque_split[2], color='y')
        plt.xlabel('time')
        plt.ylabel('Torque/Angle')
        plt.legend(['angle', 'torque'])
        plt.title("torque/angle vs time, id2 = %s"%(id2_num))
        pdf.savefig()
        plt.close()

In [126]:
for id in data_id:
    draw_pdf(id)